# SPGR

In [1]:
import version_control

Last run on cashlab03-deb at 2015-03-27 14:24:15.072125

Phypno Version: 01ade7cd0403c2fb7015744a0e2642e9dee19871

Babel==1.3, Cython==0.22, Jinja2==2.7.3, Markdown==2.6.1, MarkupSafe==0.23, Pillow==2.7.0, PyOpenGL==3.1.0, Pygments==2.0.2, Sphinx==1.3.1, alabaster==0.7.3, bash-kernel==0.3, beautifulsoup4==4.3.2, biopython==1.65, certifi==14.05.14, coverage==3.7.1, decorator==3.4.2, docutils==0.12, easydev==0.8.3, ipython==3.0.0, jedi==0.8.1-final0, jsonschema==2.4.0, matplotlib==1.4.3, memory-profiler==0.32, mistune==0.5.1, networkx==1.9.1, nibabel==2.0.0, nitime==0.5, nltk==3.0.2, nose==1.3.4, numpy==1.9.2, numpydoc==0.5, ordereddict==1.1, pandas==0.16.0, pandocfilters==1.2.3, pep8==1.6.2, pexpect==3.3, pockets==0.2.4, psutil==2.2.1, pycurl==7.19.5.1, pyflakes==0.8.1, pyparsing==2.0.3, pyqtgraph==0.9.10, python-dateutil==2.4.1, python-igraph==0.7.1-4, pytz==2015.2, pyzmq==14.5.0, requests==2.6.0, rope-py3k==0.9.4-1, rpy2==2.5.6, scikit-learn==0.16.0, scipy==0.15.1, six==1.9.0, snowbal

In [2]:
from spgr import lg

## Spindle Properties: Descriptive

### Parameters

In [3]:
hemi_subj = {'EM09': 'rh', 
             'MG17': 'rh',
             'MG33': 'lh',
             'MG37': 'lh',
             'MG61': 'lh',
             'MG63': 'rh',
             }
all_subj = sorted(hemi_subj.keys())

In [4]:
options = {'reref': 15,
           'resample_freq': 256,
           'hp_filter': 0.5,
           'lp_filter': 50,
           }
chan_type = ('grid', 'strip')

In [5]:
spindles_options = {'method': 'Nir2011', 
                    'frequency': (9, 16), 
                    'duration':(0.5, 2),
                    }
spindles_options.update(options)

In [6]:
STD = 10
THRESHOLD = 30

### Code

In [7]:
from pathlib import Path
from phypno.attr import Freesurfer
from phypno.source import Linear, Morph
from spgr.detect_spindles import get_spindles
from spgr.read_data import FS_PATH, GROUP_DIR, REC_DIR, get_chan_used_in_analysis, get_data
GROUP_DIR = Path(GROUP_DIR)
REC_DIR = Path(REC_DIR)

In [8]:
images_dir = GROUP_DIR.joinpath('images').joinpath('property_descriptive')
try:
    images_dir.mkdir()
except FileExistsError:
    pass

PLOT_COLOR = 'kw'

In [9]:
from numpy import diff, histogram, arange, argmax, median, mean, max, sum

In [ ]:
all_count = []
all_freq = []
all_val = []
all_dur = []
all_hist = []
all_peak_hist = []

for subj in all_subj:
    fs = Freesurfer(str(REC_DIR.joinpath(subj).joinpath(FS_PATH)))
    surf = getattr(fs.read_brain(), hemi_subj[subj])

    data = get_data(subj, 'sleep', chan_type, **options)
    chan = data.attr['chan']
    spindles = get_spindles(subj, chan_type=chan_type, **spindles_options)
    for one_spindle in spindles.spindle:
        one_spindle['duration'] = one_spindle['end_time'] - one_spindle['start_time']
    
    dat_count = spindles.to_data('count')
    n_min = (data.number_of('trial') * (data.axis['time'][0][-1] - data.axis['time'][0][0])) / 60
    dat_count.data[0] = dat_count.data[0] / n_min
    dat_freq = spindles.to_data('peak_freq')
    dat_val = spindles.to_data('peak_val')
    dat_dur = spindles.to_data('duration')
 
    if hemi_subj[subj] == 'lh':
        for one_chan in chan.chan:
            one_chan.xyz *= (-1, 1, 1)  # TODO: we need to improve this!
            
    surf = getattr(fs.read_brain(), 'rh')
    l = Linear(surf, chan, std=STD, threshold=THRESHOLD)
    m = Morph(surf)

    all_count.append(m(l(dat_count)))
    all_freq.append(m(l(dat_freq)))
    all_val.append(m(l(dat_val)))
    all_dur.append(m(l(dat_dur)))

INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:Subj EM09, reading data: EM09_sleep_grid-strip_hp005_lp500_rs256.pkl
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:EM09 analysis chan 77, with location 77
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:Subj MG17, reading data: MG17_sleep_grid-strip_hp005_lp500_rs256.pkl
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:MG17 analysis chan 104, with location 104
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:Subj MG17, reading data: MG17_sleep_grid-strip_hp005_lp500_rs256.pkl
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:MG17 analysis chan 104, with location 104


In [ ]:
from spgr.plot_spindles import plot_surf
from IPython.display import display

In [ ]:
v = plot_surf(all_count, threshold=(0.01, None), limits=(0, 2))
v.save(str(images_dir.joinpath('density_{}.png'.format(options['reref']))))
v

In [ ]:
v = plot_surf(all_freq, threshold=spindles_options['frequency'], limits=(11, 13))
v.save(str(images_dir.joinpath('frequency_{}.png'.format(options['reref']))))
v

In [ ]:
v = plot_surf(all_val, threshold=(0.001, None), limits=(0, 70))
v.save(str(images_dir.joinpath('amplitude_{}.png'.format(options['reref']))))
v

In [ ]:
v = plot_surf(all_dur, threshold=(0.5, 2), limits=(1, 1.5))
v.save(str(images_dir.joinpath('duration_{}.png'.format(options['reref']))))
v